# Black Scholes European Option Price Calculator

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math
import scipy.stats as sp
import yfinance as yf


In [ ]:
## Call Option

def blackScholesCall(symbol, X, S, r, T):

    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    S = todays_data['Close'][0]
    today = datetime.today()
    last_year = today - relativedelta(years=1)

    data = yf.download(symbol, start= last_year, end= today)
    data['Daily Return'] = data['Adj Close'].pct_change()
    daily_vol = data['Daily Return'].std()
    sigma = daily_vol * (252**0.5)

    d1 = (math.log(S/X) + (r + 0.5 * sigma **2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    price = S * sp.norm.cdf(d1) - X * math.exp(-r * T) * sp.norm.cdf(d2)
    
    return price

## Put Option

def blackScholesPut(symbol, X, S, r, T):
   
   ticker = yf.Ticker(symbol)
   todays_data = ticker.history(period='1d')
   S = todays_data['Close'][0]
   today = datetime.today()
   last_year = today - relativedelta(years=1)

   data = yf.download(symbol, start= last_year, end= today)
   data['Daily Return'] = data['Adj Close'].pct_change()
   daily_vol = data['Daily Return'].std()
   sigma = daily_vol * (252**0.5)
   
   d1 = (math.log(S/X) + (r + 0.5 * sigma **2) * T) / (sigma * math.sqrt(T))
   d2 = d1 - sigma * math.sqrt(T)
   
   price = X * math.exp(-r * T) * sp.norm.cdf(-d2) - S * sp.norm.cdf(-d1)

   return  price

## The Greeks

In [ ]:
## Delta

def delta_call(S, X, T, r, sigma):
  # Calculate d1 for the Call option
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  # Call option delta is N(d1)
  delta = sp.norm.cdf(d1)
  return delta

def delta_put(S, X, T, r, sigma):
  # Calculate d1 for the Put option
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  # Put option delta is -N(-d1)
  delta = -sp.norm.cdf(-d1)
  return delta

## Gamma

def gamma_call(S, X, T, r, sigma):
  
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

  gamma = sp.norm.pdf(d1) / (S * sigma * np.sqrt(T))
  return gamma

def gamma_put(S, X, T, r, sigma):
  
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  
  gamma = sp.norm.pdf(d1) / (S * sigma * np.sqrt(T))
  return gamma

## Vega

def vega_call(S, X, T, r, sigma):
  
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

  vega = S * sp.norm.pdf(d1) * np.sqrt(T)
  return vega

def vega_put(S, X, T, r, sigma):
  
  d1 = (np.log(S/X) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

  vega = S * sp.norm.pdf(d1) * np.sqrt(T)
  return vega

## Theta

def theta_call(r, S, K, T, sigma):
  
  d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  theta_call = (-S * sp.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * sp.norm.cdf(d2))

  return theta_call / 365

def theta_put(r, S, K, T, sigma):

  d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  theta_put = (-S * sp.norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * sp.norm.cdf(-d2))

  return theta_put / 365

def rho_call(r, S, K, T, sigma):
  
  d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  rho_call = K * T * np.exp(-r * T) * sp.norm.cdf(d2)
  return rho_call

def rho_put(r, S, K, T, sigma):
  d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)

  rho_put = -K * T * np.exp(-r * T) * sp.norm.cdf(-d2)

  return rho_put



## Newton-Raphson & Implied Volatility